In [1]:
clean_up = True
%run StdPackages.ipynb
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])

## Run various climate budget simulations for each year

In [2]:
t0 = 1990 # initial year
tE = 2019 # terminal year
name = 'uniform' # model/scenario name
ns = [22, 30, 'T'] # what type of time horizons for eliminating the climate debt are we using

Define standard computational settings (these are options that are passed to the solvers in ```climateBudget_t```):

In [3]:
kwargs_SY = {'name': 'SY', 'loop': 'l_SY', 'n': 10}
kwargs_IIa= {'name': 'IIa', 'loop': 'l_IIa', 'n': 10}
kwargs_IIb= {'name': 'IIb', 'loop': 'l_IIb', 'n': 3}
kwargs_IIIa = {ns[0]: {'name': f'IIIa_{ns[0]}', 'loop': f'l_IIIa_{ns[0]}', 'n': 10}}
[kwargs_IIIa.__setitem__(n, {'name': f'IIIa_{n}', 'loop': f'l_IIIa_{n}', 'n': 3}) for n in ns[1:]];
kwargs_IIIb = {ns[0]: {'name': f'IIIb_{ns[0]}', 'loop': f'l_IIIb_{ns[0]}', 'n': 10}}
[kwargs_IIIb.__setitem__(n, {'name': f'IIIb_{n}', 'loop': f'l_IIIb_{n}', 'n': 3}) for n in ns[1:]];
kwargs_IIIc = {ns[0]: {'name': f'IIIc_{ns[0]}', 'loop': f'l_IIIc_{ns[0]}', 'n': 10}}
[kwargs_IIIc.__setitem__(n, {'name': f'IIIc_{n}', 'loop': f'l_IIIc_{n}', 'n': 3}) for n in ns[1:]];

The main thing that we adjust is the number of iterations used to solve for the "Single Year" (SY) policy. This turns out to be quite tricky for the early years; thus, we pass separate options for these as follows:

In [4]:
SY_n = {n: 10 for n in range(t0,tE+1)} # use 10 as default, except if something else is specified here
SY_n.update({1990: 200,
             1991: 400,
             1992: 400,
             1993: 400,
             1994: 400,
             1995: 400,
             1996: 400,
             1997: 400,
             1998: 400,
             1999: 200,
             2000: 200,
             2001: 200,
             2002: 200,
             2003: 200,
             2014: 10,
             # 2015: 5, # include this in the 'm' model
             2015: 10,
             2016: 10,
             2017: 10,
             2018: 10,
             2019: 15}) # set 2019 = 10 for the 'm' model

Now, loop through setup for each year. For each year, this runs through 9-10 modules (parts of the general equilibrium model), combines them, and calibrates the model in general equilibrium. For 30 years, this takes quite a lot of time to run.

In [5]:
for t in range(t0,tE+1):
    kwargs_SY['n'] = SY_n[t]
    %run climateBudget_t.ipynb